In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://olympic-container@olympicstorageaccount01.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympic",
extra_configs = configs)

In [0]:
%fs
ls '/mnt/tokyoolympic'

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1700732333000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1700732342000


In [0]:
spark  #in DataBricks spark session is already loaded

In [0]:
athletes = spark.read.csv('/mnt/tokyoolympic/raw-data/athletes.csv', header=True)
coaches = spark.read.csv('/mnt/tokyoolympic/raw-data/coaches.csv', header=True)
entriesgender = spark.read.csv('/mnt/tokyoolympic/raw-data/entriesgender.csv', header=True)
medals = spark.read.csv('/mnt/tokyoolympic/raw-data/medals.csv', header=True)
teams = spark.read.csv('/mnt/tokyoolympic/raw-data/teams.csv', header=True)

In [0]:
athletes.show()
coaches.show()
entriesgender.show()
medals.show()
teams.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()


root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
entriesgender = entriesgender.withColumn('Female', entriesgender['Female'].cast('integer')) \
                             .withColumn('Male', entriesgender['Male'].cast('integer')) \
                             .withColumn('Total', entriesgender['Total'].cast('integer'))

In [0]:
entriesgender = entriesgender.withColumnRenamed('Female', 'Female_Count') \
                             .withColumnRenamed('Male', 'Male_Count')

In [0]:
entriesgender = entriesgender.dropDuplicates()

In [0]:
entriesgender = entriesgender.dropna(how='all')

In [0]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
medals = medals.withColumn('Rank', medals['Rank'].cast('integer')) \
                             .withColumn('Gold', medals['Gold'].cast('integer')) \
                             .withColumn('Silver', medals['Silver'].cast('integer')) \
                             .withColumn('Bronze', medals['Bronze'].cast('integer')) \
                             .withColumn('Total', medals['Total'].cast('integer')) \
                             .withColumn('Rank by Total', medals['Rank by Total'].cast('integer')) 
                                 

In [0]:
medals = medals.withColumnRenamed('Gold', 'Gold_Count') \
               .withColumnRenamed('Silver', 'Silver_Count') \
               .withColumnRenamed('Bronze', 'Bronze_Count') 
                   

In [0]:
medals.show()

+----+--------------------+----------+------------+------------+-----+-------------+
|Rank|        Team_Country|Gold_Count|Silver_Count|Bronze_Count|Total|Rank by Total|
+----+--------------------+----------+------------+------------+-----+-------------+
|   1|United States of ...|        39|          41|          33|  113|            1|
|   2|People's Republic...|        38|          32|          18|   88|            2|
|   3|               Japan|        27|          14|          17|   58|            5|
|   4|       Great Britain|        22|          21|          22|   65|            4|
|   5|                 ROC|        20|          28|          23|   71|            3|
|   6|           Australia|        17|           7|          22|   46|            6|
|   7|         Netherlands|        10|          12|          14|   36|            9|
|   8|              France|        10|          12|          11|   33|           10|
|   9|             Germany|        10|          11|          16| 

In [0]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
teams = teams.withColumnRenamed('Event', 'Event_Gender')

In [0]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|Event_Gender|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
athletes.write.csv('/mnt/tokyoolympic/transformed-data/athletes', header=True)
coaches.write.csv('/mnt/tokyoolympic/transformed-data/coaches',header=True)
entriesgender.write.csv('/mnt/tokyoolympic/transformed-data/entriesgender',header=True)
medals.write.csv('/mnt/tokyoolympic/transformed-data/medals',header=True)
teams.write.csv('/mnt/tokyoolympic/transformed-data/teams',header=True)